In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [ ]:
from training.xgb import main, test_xgb_model
from movie_recommender.workflow import read_parquet_from_s3, connect_minio,connect_mlflow

DB_MINIO_BUCKET = "trainingbucket"
connect_minio()
connect_mlflow()

[DEBUG] - Secure False
[DEBUG] - endpoint http://localhost:9000
[INFO] - Connected to minio on: http://localhost:9000
[DEBUG] - Available Buckets [Bucket('mlflowbucket'), Bucket('trainingbucket')]
[INFO] - Connected to mlflow on: http://localhost:8081


In [4]:
ratings = read_parquet_from_s3(DB_MINIO_BUCKET, "ratings.parquet")
movies = read_parquet_from_s3(DB_MINIO_BUCKET, "movies.parquet")
ratings = ratings.merge(movies, on="movie_id")
ratings.rating *= 5

In [5]:
booster = main(ratings,n_trials=1)

[INFO] - Running CV...
[I 2025-09-22 11:35:27,285] A new study created in memory with name: no-name-2c18af48-5e02-4bf7-9d5d-8a6ef31864f5


  0%|          | 0/1 [00:00<?, ?it/s]

[DEBUG] - Pred mean: 0.71 , std: 0.13
[DEBUG] - Pred mean: 0.71 , std: 0.13
[DEBUG] - Pred mean: 0.71 , std: 0.13
[INFO] - Found best params: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'rmse', 'booster': 'gbtree', 'max_depth': 5, 'learning_rate': 0.03276168425168626, 'subsample': 0.9600231685831186, 'colsample_bytree': 0.8523568011650867, 'gamma': 2.545352124976935, 'lambda': 9.804641899073559, 'alpha': 0.027754441871769862}


[I 2025-09-22 11:36:14,322] Trial 0 finished with value: 0.6318514347076416 and parameters: {'objective': 'reg:squarederror', 'device': 'cuda', 'random_state': 0, 'eval_metric': 'rmse', 'booster': 'gbtree', 'max_depth': 5, 'learning_rate': 0.03276168425168626, 'subsample': 0.9600231685831186, 'colsample_bytree': 0.8523568011650867, 'gamma': 2.545352124976935, 'lambda': 9.804641899073559, 'alpha': 0.027754441871769862}. Best is trial 0 with value: 0.6318514347076416.
[0]	train-rmse:0.20778	val-rmse:0.20769
[687]	train-rmse:0.15700	val-rmse:0.16974


[INFO] - Found most impactful features: ['movie_avg_rating', 'mean_Adventure', 'mean_Thriller', 'mean_Drama', 'mean_Comedy', 'user_avg_rating']
[INFO] - 

[INFO] - Training XGBMR...
[INFO] - Run id: 4fb37470b6c0491dbed256fc6a725e57


[0]	train-rmse:0.20739	train-mae:0.79966	val-rmse:0.20736	val-mae:0.79830
[500]	train-rmse:0.15838	train-mae:0.58702	val-rmse:0.17060	val-mae:0.63087
[917]	train-rmse:0.15838	train-mae:0.58703	val-rmse:0.17060	val-mae:0.63088


2025/09/22 11:36:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
[INFO] - Logging artifacts.
[DEBUG] - Logging folder /tmp/tmp04qxrjat
[INFO] - Done training.


🏃 View run xgb_2025_09_22_11h_36m_31s at: http://localhost:8081/#/experiments/1/runs/4fb37470b6c0491dbed256fc6a725e57
🧪 View experiment at: http://localhost:8081/#/experiments/1


Registered model 'xgb' already exists. Creating a new version of this model...
2025/09/22 11:36:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb, version 4
Created version '4' of model 'xgb'.
[DEBUG] - Champion 'val-mae': 0.6314, Latest 'val-mae': 0.6309
[DEBUG] - Promoted version='4' of registered_name='xgb' to champion!
[DEBUG] - 'xgb' has new champion
[INFO] - 

[INFO] - Saving plots...
[INFO] - True  : mean 3.557, std 1.051
[INFO] - Pred  : mean 3.557, std 0.671
[INFO] - MAE   : 0.5870039341768053
[INFO] - Error : mean 0.000, std 0.792
[INFO] - Test
[INFO] - True  : mean 3.557, std 1.049
[INFO] - Pred  : mean 3.557, std 0.663
[INFO] - MAE   : 0.6308777166141769
[INFO] - Error : mean 0.000, std 0.853
[DEBUG] - Logging folder /tmp/tmp8j0p0cut


In [6]:
test_xgb_model()

[INFO] - XGB loaded on device: cuda
[DEBUG] - Getting artifacts, run_id='4fb37470b6c0491dbed256fc6a725e57', artifact_path='resources'


[INFO] - Loaded champion model, registered_name='xgb'
[INFO] - XGB model test passed successfully
